In [ ]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import torch
from glob import glob

import efficientnet
import efficientnet.tfkeras as efn

from tensorflow.keras.models import load_model
from keras.preprocessing import image

from IPython.display import Image

In [ ]:
new_model = load_model('./e_final_model_5.h5')

### 예측할 이미지 불러오기

In [ ]:
img_list = glob('/home/lab30/jupyter_home/test/*.jpg')

print(len(img_list)) # 모든 이미지의 개수 확인

8


In [ ]:
cd /home/lab30/jupyter_home/yolov5

/home/lab30/jupyter_home/yolov5


In [ ]:
os.environ['MKL_THREADING_LAYER'] = 'GNU'

img_path = img_list[1]

!python detect.py --weights ./runs/train/best.pt --img 416 --conf 0.5 --source "{img_path}" --save-crop

detect: weights=['./runs/train/best.pt'], source=/home/lab30/jupyter_home/test/es_img_1 (5).jpg, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-246-g2dd3db0 Python-3.8.13 torch-1.11.0+cu102 CUDA:0 (Tesla T4, 15110MiB)

Traceback (most recent call last):
  File "detect.py", line 252, in <module>
    main(opt)
  File "detect.py", line 247, in main
    run(**vars(opt))
  File "/home/ubuntu/anaconda3/envs/project_cnn/lib/python3.8/site-packages/torch/autograd/grad_mode.py", line 27, in decorate_context
    return func(*args, **kwargs)
  File "detect.py", line 92, in run
    model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=h

###  crop된 이미지 불러오기

In [ ]:
# detect 하위 dir 경로 및 해당 dir name (concat)
subfolders = pd.DataFrame([ f.path for f in os.scandir('/home/lab30/jupyter_home/yolov5/runs/detect/') if f.is_dir() ], columns = ['dir_path'])
subfolders_nm = pd.DataFrame([ f.name for f in os.scandir('/home/lab30/jupyter_home/yolov5/runs/detect/') if f.is_dir() ], columns = ['dir_name'])
subfolders_frm = pd.concat([subfolders, subfolders_nm], axis=1)

# checkpoints dir 제거(사용할 최종 data) : sub_exp_frm
sub_exp_frm = subfolders_frm.loc[~subfolders_frm.dir_name.str.contains('check'),:]

# exp 뒤의 숫자가 가장 큰 dir 찾기 (가장 최신 detection) : new_dir
new_idx = sub_exp_frm.dir_name.str[3:].replace('',0).astype(int).argmax() # 해당 dir 있는 index number
new_dir = sub_exp_frm.iloc[new_idx, 0]

if 'crops' in os.listdir(new_dir) :
    for (path, dir, files) in os.walk(new_dir + '/crops/') :
        for filename in files :
            ext = os.path.splitext(filename)[-1]
            if ext == '.JPG' or ext == '.jpg' :
                label_nm = path.split('/')[-1]
                crop_img_path = '%s/%s'% (path, filename)
                print(label_nm)
                print(crop_img_path)
else :
    print("No crop image")

disable1
/home/lab30/jupyter_home/yolov5/runs/detect/exp/crops/disable1/crop4.jpg


### predict

In [ ]:
import numpy as np
from keras.preprocessing import image
    

test_image = image.load_img(crop_img_path, target_size = (150, 150))

test_image = image.img_to_array(test_image)

test_image = np.expand_dims(test_image, axis = 0)

result = new_model.predict(test_image)

np.max(result)


0.9874752